In [1]:
from functions import Model, get99, reg_train_test, splitData, dropMissingValues, z_score, outliers_knn, outliers_dbscan, getNoise, getRelFeatures, drop_features, getCombinations, get_unique_list, bold, blue, red, green, getBestModel, np_to_df, df_to_np, drop99_all
import pandas as pd
from sklearn.linear_model import LinearRegression
import warnings
import json
import numpy as np
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

In [2]:
# JSON file
f = open ('data/json/input.json', "r")
  # Reading from file
PARAMETERS = json.loads(f.read())

SPLIT_SIZE = PARAMETERS["SPLIT_SIZE"]

OUTLIER99 = PARAMETERS["OUTLIER99"] 
OUTLIER_ZSCORE = {"STD" : PARAMETERS["OUTLIER_ZSCORE"]["STD"]}
OUTLIER_KNN = {"K": PARAMETERS["OUTLIER_KNN"]["K"], "X": PARAMETERS["OUTLIER_KNN"]["X"]}
OUTLIER_DBSCAN = {"K": PARAMETERS["OUTLIER_DBSCAN"]["K"], "X": PARAMETERS["OUTLIER_DBSCAN"]["X"], "EPS": PARAMETERS["OUTLIER_DBSCAN"]["EPS"], "MIN_SAMPLES":  PARAMETERS["OUTLIER_DBSCAN"]["MIN_SAMPLES"]}
OUTLIER_NOISE = {"CV": PARAMETERS["OUTLIER_NOISE"]["CV"]}

FEATURE_SEL_CORR = {"THRESHOLD_CORR": PARAMETERS["FEATURE_SEL_CORR"]["THRESHOLD"]}

f.close()

#TODO FEATURE
#TODO Regressionmodels
#TODO TESTDATA --> path to csv-file


In [3]:
#Get raw Data
df = pd.read_csv("data/csv/house_data_training.csv", sep=';') 
# remove unnamed column
df = df.iloc[:, 1:]
#Transform string to datetime
df["date"] = pd.to_datetime(df["date"])

maeList = []#for model selection

In [4]:
df = dropMissingValues(df) 

In [5]:
# Outlier Detection
outlier_dict_all = {}

if OUTLIER99 == True:
    print("outlier_99")
    outlier_list_99 = get99(df) #TODO Mean for price for trainingsdata
    df = drop99_all(df, outlier_list_99 )
    #outlier_dict_all["99_mean"] = outlier_list_99
    #outlier_dict_all["99_drop"] = outlier_list_99
    print("done\n")

print('starting z_score')
for p in OUTLIER_ZSCORE["STD"]:
    print(p)
    outlier_list_z_score = z_score(df, p)
    outlier_dict_all["z_score_"+str(p)] = outlier_list_z_score

print('done \n\nstarting knn')
for k in OUTLIER_KNN["K"]:
    for o in OUTLIER_KNN["X"]:
        print(k, o)
        outlier_list_knn = outliers_knn(df, k, o)
        outlier_dict_all["knn_"+str(k)+"_"+str(o)] = outlier_list_knn

print('done \n\nstarting db_scan')
outlier_list_dbscan = outliers_dbscan(df) #TODO multiple parameters
outlier_dict_all["db_scan"] = outlier_list_dbscan

print('done \n\nstarting noise')
for cv in OUTLIER_NOISE["CV"]:
        print(cv)
        outlier_list_noise = getNoise(df, cv)
        outlier_dict_all["noise_"+str(cv)] = outlier_list_noise
print('done')

outlier_99
done

starting z_score
3
10
done 

starting knn
3 200
3 1000
5 200
5 1000
done 

starting db_scan
done 

starting noise
5
10
done


In [6]:
outlier_lists_all = list(outlier_dict_all.values())
outlier_lists_keys = list(outlier_dict_all.keys())
combination_list = []
for i, combo in enumerate(getCombinations(outlier_lists_keys), 1):
    if not i == 1: #emtpy tuple
        combination_list.append(combo)
        # print('combo #{}: {}'.format(i, combo))
print("...")
for c in combination_list[10:]:
    print(bold("Combination: "), blue(c),"\n",)
print(bold(len(combination_list)), " total Combinations")

...
Combination:  ('z_score_3', 'knn_3_200') 

Combination:  ('z_score_3', 'knn_3_1000') 

Combination:  ('z_score_3', 'knn_5_200') 

Combination:  ('z_score_3', 'knn_5_1000') 

Combination:  ('z_score_3', 'db_scan') 

Combination:  ('z_score_3', 'noise_5') 

Combination:  ('z_score_3', 'noise_10') 

Combination:  ('z_score_10', 'knn_3_200') 

Combination:  ('z_score_10', 'knn_3_1000') 

Combination:  ('z_score_10', 'knn_5_200') 

Combination:  ('z_score_10', 'knn_5_1000') 

Combination:  ('z_score_10', 'db_scan') 

Combination:  ('z_score_10', 'noise_5') 

Combination:  ('z_score_10', 'noise_10') 

Combination:  ('knn_3_200', 'knn_3_1000') 

Combination:  ('knn_3_200', 'knn_5_200') 

Combination:  ('knn_3_200', 'knn_5_1000') 

Combination:  ('knn_3_200', 'db_scan') 

Combination:  ('knn_3_200', 'noise_5') 

Combination:  ('knn_3_200', 'noise_10') 

Combination:  ('knn_3_1000', 'knn_5_200') 

Combination:  ('knn_3_1000', 'knn_5_1000') 

Combination:  ('knn_3_1000', 'db_scan') 

Combina

In [7]:
model_obj_list = []
for i, c in enumerate(combination_list):
    print(f'\n{bold(i)}: {blue(c)}')
    outlier_list = get_unique_list(outlier_dict_all, c)
    try: 
        if "99_mean" in c:
            X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "both", outlier_list_99)
        else:
            X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "drop")

        #TODO Regressionmodels
        model = LinearRegression()
        obj = Model(model, (X_train, X_test, y_train, y_test), c, df)
        model_obj_list.append(obj)

    except Exception as e:
        print(bold("Error"), red(e))


0: ('z_score_3',)
dropped 1025 / 1357 rows
replaced 0 / 1357 rows

1: ('z_score_10',)
dropped 42 / 54 rows
replaced 0 / 54 rows

2: ('knn_3_200',)
Error 'numpy.ndarray' object has no attribute 'remove'

3: ('knn_3_1000',)
Error 'numpy.ndarray' object has no attribute 'remove'

4: ('knn_5_200',)
Error 'numpy.ndarray' object has no attribute 'remove'

5: ('knn_5_1000',)
Error 'numpy.ndarray' object has no attribute 'remove'

6: ('db_scan',)
dropped 2530 / 2682 rows
replaced 0 / 2682 rows

7: ('noise_5',)
dropped 35 / 51 rows
replaced 0 / 51 rows

8: ('noise_10',)
dropped 39 / 59 rows
replaced 0 / 59 rows

9: ('z_score_3', 'z_score_10')
dropped 1025 / 1025 rows
replaced 0 / 1025 rows

10: ('z_score_3', 'knn_3_200')
dropped 1202 / 1208 rows
replaced 0 / 1208 rows

11: ('z_score_3', 'knn_3_1000')
dropped 1875 / 1921 rows
replaced 0 / 1921 rows

12: ('z_score_3', 'knn_5_200')
dropped 1202 / 1208 rows
replaced 0 / 1208 rows

13: ('z_score_3', 'knn_5_1000')
dropped 1875 / 1921 rows
replaced 0

In [9]:
#Summary
df_summary = pd.DataFrame(columns=['combo', 'mae', 'score'])
for o in model_obj_list:
    #print(f'{o.get_type()}, {o.get_comb()}')
    o.train()
    mae, score = o.summary()
    comb = o.get_comb()
    row = {'combo': comb, 'mae': mae, 'score': score}
    df_summary = df_summary.append(row, ignore_index=True)

df_summary.sort_values("mae")

,combo,mae,score
1,"(z_score_10,)",165149.72,0.525593
19,"(z_score_10, noise_10)",165180.06,0.525585
4,"(noise_10,)",165180.47,0.525626
70,"(z_score_10, knn_3_200, knn_5_200)",165187.68,0.525711
15,"(z_score_10, knn_5_200)",165187.68,0.525711
...,...,...,...
402,"(z_score_3, z_score_10, knn_3_1000, knn_5_200,...",166178.32,0.525169
395,"(z_score_3, z_score_10, knn_3_200, knn_5_1000,...",166178.32,0.525169
386,"(z_score_3, z_score_10, knn_3_200, knn_3_1000,...",166178.32,0.525169
420,"(z_score_3, knn_3_200, knn_3_1000, knn_5_1000,...",166178.32,0.525169


In [10]:
#row
mae_best = df_summary[df_summary.mae==df_summary.mae.min()]
score_best = df_summary[df_summary.score==df_summary.score.max()]
best_model_obj = getBestModel(model_obj_list, df_summary, mae_best.index[0])
print(best_model_obj)
print(f'{bold("Lowest mae:")} {mae_best}')

Model: LinearRegression() Combination: ('z_score_10',) Features: 24
Lowest mae:            combo        mae     score
1  (z_score_10,)  165149.72  0.525593


In [11]:
model_obj_list_best = [best_model_obj]
df_feature_reduction = best_model_obj.get_df().head()

# print(df_feature_reduction)

for threshold in FEATURE_SEL_CORR["THRESHOLD_CORR"]:
    #print(threshold, type(threshold))
    try: 
        rel_features = getRelFeatures(df_feature_reduction, threshold)
        #print(len(rel_features), rel_features)
        df_feature_reduction = df_feature_reduction[rel_features]
        X_train, X_test, y_train, y_test = splitData(df_feature_reduction, SPLIT_SIZE)
        best_model_obj_new = Model(best_model_obj.get_model(), (X_train, X_test, y_train, y_test), best_model_obj.get_comb() , df_feature_reduction)
        model_obj_list_best.append(best_model_obj_new)

    except Exception as e:
        print(bold("Error"), red(e))
print(model_obj_list_best)

price
dis_super
sqft_living15
view
zipcode
grade
sqft_living
bathrooms
sqft_lot
lat
sqft_lot15
floors
sqft_basement
sqft_above
ahf1
id
yr_built
ahf3
condition
long
bedrooms
ahf2
waterfront
yr_renovated
price
dis_super
sqft_living15
view
zipcode
grade
sqft_living
bathrooms
sqft_lot
sqft_lot15
lat
ahf1
id
yr_built
ahf3
condition
long
bedrooms
ahf2
price
dis_super
sqft_living15
view
zipcode
grade
sqft_living
bathrooms
sqft_lot
long
bedrooms
ahf2
[Model: LinearRegression() Combination: ('z_score_10',) Features: 24, Model: LinearRegression() Combination: ('z_score_10',) Features: 18, Model: LinearRegression() Combination: ('z_score_10',) Features: 11, Model: LinearRegression() Combination: ('z_score_10',) Features: 10]


In [12]:
#Summary
df_summary_best = pd.DataFrame(columns=['combo', 'mae', 'score', 'features'])
for o in model_obj_list_best:
    #print(f'{o.get_type()}, {o.get_comb()}')
    o.train()
    mae, score = o.summary()
    comb = o.get_comb()
    features = o.get_df().columns.to_list()
    features.remove("price")
    row = {'combo': comb, 'mae': mae, 'score': score, 'features': features}
    df_summary_best = df_summary_best.append(row, ignore_index=True)

df_summary_best.sort_values("mae")

,combo,mae,score,features
0,"(z_score_10,)",165149.72,0.525593,"[id, date, bedrooms, bathrooms, sqft_living, s..."
3,"(z_score_10,)",633389.13,-0.391766,"[dis_super, sqft_living15, view, zipcode, grad..."
2,"(z_score_10,)",634736.26,-0.397251,"[dis_super, sqft_living15, view, zipcode, grad..."
1,"(z_score_10,)",713630.29,-0.771741,"[dis_super, sqft_living15, view, zipcode, grad..."


In [13]:
#row
mae_best = df_summary_best[df_summary_best.mae==df_summary_best.mae.min()]
score_best = df_summary_best[df_summary_best.score==df_summary_best.score.max()]
best_model_obj = getBestModel(model_obj_list_best, df_summary_best, mae_best.index[0])
print(best_model_obj)
print(f'{bold("Lowest mae:")} {mae_best}')


Model: LinearRegression() Combination: ('z_score_10',) Features: 24
Lowest mae:            combo        mae     score  \
0  (z_score_10,)  165149.72  0.525593   

                                            features  
0  [id, date, bedrooms, bathrooms, sqft_living, s...  


In [14]:
# def getDF(self):
#     X_train, X_test, y_train, y_test = self.get_data()
#     print(len(y_test))
#     #print(y_train.shape, y_test.reshape(len(y_test), 1).shape)
#     X = np.append(X_train, X_test, axis=0)
#     y = np.append(y_train, y_test.reshape(len(y_test), 1), axis=0)
#     X_y = np.append(X, y, axis=1)
#     #....

# df_feature = best_model_obj.get_df()

In [15]:
# combination_list = []
# df_feature_reduction = best_model_obj.get_df()

# for i, combo in enumerate(getCombinations(df_feature_reduction.columns.to_list()), 1):
#     if not i == 1: #emtpy tuple
#         combination_list.append(combo)
#         # print('combo #{}: {}'.format(i, combo))
# print("...")
# for c in combination_list[10:]:
#     print(bold("Combination: "), blue(c),"\n",)
# print(bold(len(combination_list)), " total Combinations")

In [16]:
# model_obj_list_best = []
# df_feature_reduction = best_model_obj.get_df()
# for i, c in enumerate(combination_list):
#     print(f'\n{bold(i)}: {blue(c)}')
    
#     try: 
#         X_train, X_test, y_train, y_test = splitData(df_feature_reduction, SPLIT_SIZE)
#         best_model_obj_copy = best_model_obj.copy()
#         best_model_obj_copy.set_data(X_train, X_test, y_train, y_test)

#         model_obj_list_best.append(obj)

#     except Exception as e:
#         print(bold("Error"), red(e))

In [17]:
# df_feature_reduction = best_model_obj.get_df()
# getCombinations(df_feature_reduction.columns.to_list())

# for feature in list(df_feature_reduction.columns.to_list()):
#     df_feature_reduction.drop(columns=feature)
#     X_train, X_test, y_train, y_test = splitData(df_feature_reduction, SPLIT_SIZE)
#     best_model_obj_copy = best_model_obj.copy()
#     best_model_obj_copy.set_data(X_train, X_test, y_train, y_test)

In [18]:
# #Wrapper approaches
# #Backward elimination using Recursive feature elimination¶
# from sklearn.svm import SVC
# from sklearn import feature_selection
# import pandas as pd
# import numpy as np


# x, y = df.drop(columns=["price"]), df["price"]
# # x_y= np.concatenate((x.reshape(150,4),y.reshape(150,1)),1)
# x_y= np.concatenate((x,y),1)
# #print(x_y)
# #print(iris.target_names)

# #Create column list
# df_columns=df.columns
# # df_columns.append("Label")
# # #Create PANDAS data frame
# # df = pd.DataFrame(x_y,columns=df_columns)
# #Map label index to label name
# df['price']=df['price'].map(lambda x: iris.target_names[int(x)])
# print("\nOriginal Iris Data Set:")
# print(df)

# #Create the RFE object and rank features
# num_features=2
# svc = SVC(kernel="linear", C=1)
# rfe = feature_selection.RFE(estimator=svc, n_features_to_select=num_features, step=1)
# rfe.fit(x, y)
# #print("Selected features will have a ranking=1 and support=TRUE")
# #print(iris.feature_names," ",rfe.ranking_," ",rfe.support_)
# #print(x.shape)

# #extend column-mask by one column for the label (always true)
# column_mask=np.append(rfe.support_,True)
# #use list column_mask to mask df-columns list
# reduced_iris_features = [df_columns[i] for i in range(len(df_columns)) if column_mask[i]]
# print("Reduced_iris_features: ",reduced_iris_features)
# # use reduced_iris_features to reduce the data frame
# reduced_df=df[reduced_iris_features]

# print("\nIris Data Set reduced to ",num_features," features: \n",reduced_df)

In [19]:
#TODO sqft_living wenn price auffällig niedrig obwohl m² hoch clsuter
#TODO PCA macht warscheinlich wenig sinn